# Viterbi Algorithm

Useful resources:

* Wikipedia article with example: https://en.wikipedia.org/wiki/Viterbi_algorithm
* Viterbi Algorithm Clarified (same example as wikipedia): http://blog.ivank.net/viterbi-algorithm-clarified.html
* Short explanation and runtime behaviour: http://www.cs.toronto.edu/~sengels/tutorials/viterbi.html
* A Tutorial on Convolutional Coding with Viterbi Decoding: http://home.netcom.com/~chip.f/viterbi/tutorial.html
* https://jyyuan.wordpress.com/2014/01/22/viterbi-algorithm-finding-most-likely-sequence-in-hmm/

This notebook is using the example from the above mentioned wikipedia article. The viterbi algorithm is used in the decoding step when used in conjunction with Hidden Markov Models (HMMs).


## Input

* The **observation space** $O = \{o_1, o2,...,o_N\}$
* The **state space** $S = \{ s_1, s_2,...,s_K\}$
* An array of **initial probabilities** $\Pi = ( \pi_1, \pi_2,...,\pi_K )$ such that $\pi_i$ stores the probability that $x_1 == s_i$
* A **sequence of observations** $Y = (y_1,y_2,...,y_T)$ such that $y_t == i$ if the observation at time $t$ is $o_i$
* **Transition probability matrix A** of size $K \times K$ such that $A_{ij}$ stores the transition probability of transiting from state $s_i$ to state $s_j$
* **Observation likelihoods / Emission probability matrix B** of size $K \times N$ such that $B_{ij}$ stores the probability of observing $o_j$ from state $s_i$



## Output

* The most likely hidden state sequence $X = (x_1,x_2,...,x_N)$


## Doctor example

Consider a village where all villagers are either **healthy** or have a **fever** and only the village doctor can determine whether each has a fever. The doctor diagnoses fever by asking patients how they feel. The villagers may only answer that they feel **normal, dizzy, or cold**.

The doctor believes that the health condition of his patients operate as a discrete Markov chain. There are two states, **"Healthy" and "Fever"**, but the doctor cannot observe them **directly**; they are **hidden** from him. On each day, there is a certain chance that the patient will tell the doctor he/she is **"normal", "cold", or "dizzy"**, depending on their health condition.

The observations (normal, cold, dizzy) along with a hidden state (healthy, fever) form a hidden Markov model (HMM).

![HMM Example](images/An_example_of_HMM.png)

In [2]:
# *** Observations/Emissions ***
#
# What you can actually measure/see/observe
obs = ('normal', 'cold', 'dizzy')


# *** Hidden states ***
#
# States which you cannot measure/see/observe
# and cause the observations/emissions.
# Same concept as latent variables?
states = ('Healthy', 'Fever')

# ** Start probability ***
#
# start_probability represents the doctor's belief about 
# which state the HMM is in when the patient first visits 
# (all he knows is that the patient tends to be healthy). 
# The particular probability distribution used here is not 
# the equilibrium one, which is (given the transition 
# probabilities) approximately {'Healthy': 0.57, 'Fever': 0.43}
start_p = {'Healthy': 0.6, 'Fever': 0.4}

# *** Transition probability***
#
#
#
trans_p = {
   'Healthy' : {'Healthy': 0.7, 'Fever': 0.3},
   'Fever' : {'Healthy': 0.4, 'Fever': 0.6}
   }

# *** Emission probability***
# 
#
#
emit_p = {
   'Healthy' : {'normal': 0.5, 'cold': 0.4, 'dizzy': 0.1},
   'Fever' : {'normal': 0.1, 'cold': 0.3, 'dizzy': 0.6}
   }

In [6]:
def viterbi(obs, states, start_p, trans_p, emit_p):
    V = [{}]
    for st in states:
        V[0][st] = {"prob": start_p[st] * emit_p[st][obs[0]], "prev": None}
    # Run Viterbi when t > 0
    for t in range(1, len(obs)):
        V.append({})
        for st in states:
            max_tr_prob = max(V[t-1][prev_st]["prob"]*trans_p[prev_st][st] for prev_st in states)
            for prev_st in states:
                if V[t-1][prev_st]["prob"] * trans_p[prev_st][st] == max_tr_prob:
                    max_prob = max_tr_prob * emit_p[st][obs[t]]
                    V[t][st] = {"prob": max_prob, "prev": prev_st}
                    break
    for line in dptable(V):
        print(line)
    opt = []
    # The highest probability
    max_prob = max(value["prob"] for value in V[-1].values())
    previous = None
    # Get most probable state and its backtrack
    for st, data in V[-1].items():
        if data["prob"] == max_prob:
            opt.append(st)
            previous = st
            break
    # Follow the backtrack till the first observation
    for t in range(len(V) - 2, -1, -1):
        opt.insert(0, V[t + 1][previous]["prev"])
        previous = V[t + 1][previous]["prev"]

    print('The steps of states are ' + ' '.join(opt) + ' with highest probability of %s' % max_prob)

def dptable(V):
    # Print a table of steps from dictionary
    yield " ".join(("%12d" % i) for i in range(len(V)))
    for state in V[0]:
        yield "%.7s: " % state + " ".join("%.7s" % ("%f" % v[state]["prob"]) for v in V)

In [7]:
viterbi(obs,
        states,
        start_p,
        trans_p,
        emit_p)

           0            1            2
Healthy: 0.30000 0.08400 0.00588
Fever: 0.04000 0.02700 0.01512
The steps of states are Healthy Healthy Fever with highest probability of 0.01512


In [11]:
# Copied from: https://github.com/WuLC/ViterbiAlgorithm/blob/master/Viterbi.py

def viterbi_2(obs, states, s_pro, t_pro, e_pro):
    path = { s:[] for s in states} # init path: path[s] represents the path ends with s
    curr_pro = {}
    for s in states:
        curr_pro[s] = s_pro[s]*e_pro[s][obs[0]]
    for i in range(1, len(obs)):
        last_pro = curr_pro
        curr_pro = {}
        for curr_state in states:
            max_pro, last_sta = max(((last_pro[last_state]*t_pro[last_state][curr_state]*e_pro[curr_state][obs[i]], last_state) 
                                       for last_state in states))
            curr_pro[curr_state] = max_pro
            path[curr_state].append(last_sta)

    # find the final largest probability
    max_pro = -1
    max_path = None
    for s in states:
        path[s].append(s)
        if curr_pro[s] > max_pro:
            max_path = path[s]
            max_pro = curr_pro[s]
        # print '%s: %s'%(curr_pro[s], path[s]) # different path and their probability
    return max_path

In [12]:
print(viterbi_2(obs, states, start_p, trans_p, emit_p))

['Healthy', 'Healthy', 'Fever']


This reveals that the observations ['normal', 'cold', 'dizzy'] were most likely generated by states ['Healthy', 'Healthy', 'Fever']. In other words, given the observed activities, the patient was most likely to have been healthy both on the first day when he felt normal as well as on the second day when he felt cold, and then he contracted a fever the third day.

The operation of Viterbi's algorithm can be visualized by means of a trellis diagram. The Viterbi path is essentially the shortest path through this trellis. The trellis for the clinic example is shown below; the corresponding Viterbi path is in bold:

![HMM Example](images/Viterbi_animated_demo.gif)